# Document Classification Pipeline

In [ ]:
import psutil
import os
from pytictoc import TicToc
import pandas as pd
from documentProcessor import processAllDocuments, getLogger, fetchFiles

In [ ]:
logger = getLogger()
t = TicToc()
core_constraint = [2, 3, 4, 5, 6, 7]
psutil.Process(os.getpid()).cpu_affinity(core_constraint)
logger.debug(f"Initializing Document Classification Pipeline")
logger.info(f"CPU affinity set to cores {core_constraint}")

In [ ]:
#registerExtensions()
#analyzer = UnsupervisedDocumentAnalyzer()
#classifier = supervisedDocumentAnalyzer()
#logger.info('DocumentAnalyzer initialized')

In [ ]:
fsExamples = 'fewShot.csv'
folder = 'testDir' # Set the folder to process
directory = 'data/' + folder + '/'
tif_files = fetchFiles(directory)

In [ ]:
t.tic()
logger.info(f'Starting document processing for {len(tif_files)} files in {directory}')

results_df = processAllDocuments(tif_files, fsExamples, force_reload=False)

elapsed_time = t.tocvalue()
logger.info(f'Finished document processing for {len(tif_files)} files in {directory}. Time taken: {elapsed_time:.2f} seconds.')

In [ ]:
""" print(f'Total documents processed: {len(results_df)}')
print(f'Successfully processed: {results_df['status'].value_counts().get("success", 0)}')
print(f'Failed to process: {results_df['status'].value_counts().get("error", 0)}')

print(f'Category distribution:')
print(results_df[results_df['status'] == 'success']['category'].value_counts())

print(f'Average confidence: {results_df[results_df['status'] == 'success']['confidence'].mean():.2f}') """

In [ ]:
if 'error' in results_df['status'].values:
  print('\nError Analysis:')
  error_df = results_df[results_df['status'] == 'error']
  print(error_df['error_message'].value_counts())

  print('\nFiles with empty extracted text:')
  empty_text_files = error_df[error_df['error_message'] == 'ValueError: Extracted text is empty']
  print(empty_text_files['file_path'].tolist())

  print('\nFiles with other errors:')
  other_error_files = error_df[error_df['error_message'] != 'ValueError: Extracted text is empty']
  for _, row in other_error_files.iterrows():
    print(f'File: {row["file_path"]}')
    print(f'Error: {row["error_message"]}')
    print(f'Extracted text length: {row['text_length']}')
    print()

In [ ]:
results_df.head(15)

In [ ]:
results_df.value_counts('category')

In [ ]:
test = pd.read_csv('fewShot.csv')
test.value_counts('category')

### Check if a document is referral

In [ ]:
def isReferral(text):
  referral_keywords = ['referral to', 'referral communication form', 'end of referral report', 'patient demo']
  return any(keyword in text.lower() for keyword in referral_keywords)

In [ ]:
""" def summarize(self, text):
    try:
      if not text or len(text) < 30:
        logger.warning(f'Text length is less than 30 characters. Skipping summarization')
        return text
      
      max_tokens = self.summarizer.tokenizer.model_max_length - 100
      encoded_text = self.summarizer.tokenizer.encode(text, truncation=True, max_length=max_tokens)
      decoded_text = self.summarizer.tokenizer.decode(encoded_text, skip_special_tokens=True)
      logger.debug(f'Attempting to summarize text of length {len(decoded_text)}')

      chunk_size = 1024
      chunks = [decoded_text[i:i+chunk_size] for i in range(0, len(decoded_text), chunk_size)]

      summaries = []
      for chunk in chunks:
        chunk_summary = self.summarizer(chunk, max_length=150, min_length=30, do_sample=False)
        summaries.append(chunk_summary[0]['summary_text'])

      final_summary = " ".join(summaries)
      if len(final_summary.split()) > 150:
        final_summary = self.summarizer(final_summary, max_length=150, min_length=30, do_sample=False)[0]['summary_text']

      logger.debug(f'Summary successful. Summary length: {len(final_summary)} characters')
      return final_summary
    
    except Exception as e:
      logger.error(f'Error in summarization: {str(e)}')
      return text """